In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import json
import glob



In [2]:
files = glob.glob('./city=Nashville/year=*/month=*/*.parquet')
agg_df = []
for fp in files:
    read_df = pd.read_parquet(fp)
    read_df = read_df['ACCIDENT' == read_df['type']]
    agg_df.append(read_df)
incidents = pd.concat(agg_df)
incidents = incidents.sort_values('ts')
incidents['datetime'] = incidents['ts']
incidents.drop(['ts'], axis=1)


,confidence,country,geo,magvar,nThumbsUp,Reliability,reportRating,roadType,street,subtype,type,uuid,ts_utc,dayofweek,original_city,datetime
8751,5,US,POINT(-86.80638 36.170188),79,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,ACCIDENT,cec392ff-8cfc-3bd0-a287-b08e627462c3,2019-04-22 12:06:58,Mon,"Nashville, TN",2019-04-22 12:06:58 UTC
3266,3,US,POINT(-86.776219 36.208805),186,0.0,10.0,1,3.0,I-65 S,,ACCIDENT,a7062075-2b55-30d5-ba11-82eb9f2f1103,2019-04-22 12:08:55,Mon,"Nashville, TN",2019-04-22 12:08:55 UTC
6521,0,US,POINT(-86.776231 36.208708),186,0.0,6.0,1,3.0,I-65 S,,ACCIDENT,e2923910-7a53-3b62-893e-6f0cc55d05f5,2019-04-22 12:09:15,Mon,"Nashville, TN",2019-04-22 12:09:15 UTC
9873,0,US,POINT(-86.856853 36.321678),107,0.0,6.0,5,3.0,I-24 E,ACCIDENT_MINOR,ACCIDENT,aa4b1b43-3b3a-3040-8c34-a055a6ebb37c,2019-04-22 12:18:58,Mon,"Nashville, TN",2019-04-22 12:18:58 UTC
4360,5,US,POINT(-86.769845 36.109318),331,0.0,10.0,1,3.0,I-65 N,ACCIDENT_MINOR,ACCIDENT,2207d6c2-445a-3149-b240-8c7d970cfe71,2019-04-22 12:28:42,Mon,"Nashville, TN",2019-04-22 12:28:42 UTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,0,US,POINT(-86.72804 36.109408),102,NaN,6.0,3,6.0,Thompson Ln,ACCIDENT_MAJOR,ACCIDENT,2cf788a5-4961-4a6a-a9c7-9bb5bfd972c3,2022-03-06 21:57:19,Sun,"Nashville, TN",2022-03-06 21:57:19 UTC
5638,0,US,POINT(-86.764631 36.166673),245,NaN,5.0,3,6.0,Shelby Ave,,ACCIDENT,c1709449-57d6-4b33-8249-6f170cedc0f8,2022-03-06 22:03:01,Sun,"Nashville, TN",2022-03-06 22:03:01 UTC
19134,0,US,POINT(-86.690228 36.303853),230,NaN,5.0,1,6.0,Gallatin Pike N,ACCIDENT_MINOR,ACCIDENT,125dfe6c-6bd6-4b10-b4ec-827ffac16060,2022-03-06 22:13:03,Sun,"Nashville, TN",2022-03-06 22:13:03 UTC
13278,0,US,POINT(-86.758768 36.232374),7,NaN,5.0,0,6.0,Dickerson Pike,,ACCIDENT,58ab36e3-5d46-40a7-81b8-401a6e4a2496,2022-03-06 22:19:52,Sun,"Nashville, TN",2022-03-06 22:19:52 UTC


In [3]:
#Locate the location of accident and create a buffer for each accident
tnESPG = 'EPSG:2274'
size = 20
incidents_project = incidents
incidents_project['geometry'] = gpd.GeoSeries.from_wkt(incidents['geo'], crs='EPSG:4326')
incidents_project = gpd.GeoDataFrame(incidents_project, geometry='geometry').to_crs(tnESPG)
incidents_project['geo_buffer'] = incidents_project.geometry.buffer(size)  
incidents_project = incidents_project.set_geometry('geo_buffer')


In [12]:
#Find the Dacidson county's geometry information

Nashville = gpd.read_file('USA_Tennessee.geojson')
Nashville = Nashville[Nashville['FRC']=='1']
DAVIDSON = Nashville[Nashville['County']=='DAVIDSON'].to_crs('EPSG:2274')
DAVIDSON['space'] = DAVIDSON.geometry
DAVIDSON

,OID,XDSegID,PreviousXD,NextXDSegI,FRC,ToExport,Miles,Lanes,RoadNumber,RoadName,...,PrimaryTmc,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID,geometry
15600,8141146,160197227,1.524414e+09,4.414164e+08,1,None,0.555288,2.000,24,I-24 W,...,121+04213,I-24,36.328638,-86.89320,36.332536,-86.901924,N,1743763,4326,"LINESTRING (1705389.461 727514.661, 1702830.38..."
17840,8150357,160666629,1.524427e+09,1.607364e+08,1,None,0.484764,5.213,65,I-65 S,...,121-04246,I-65,36.110930,-86.77135,36.104830,-86.767060,S,1752974,4326,"LINESTRING (1740643.611 647949.561, 1740730.15..."
18182,8151708,160736437,1.606666e+08,1.524512e+09,1,None,0.322809,5.000,65,I-65 S,...,121N04246,I-65,36.104830,-86.76706,36.100610,-86.764610,S,1752974,4326,"LINESTRING (1741896.241 645719.975, 1741917.93..."
25097,9263459,396070012,1.524356e+09,1.524541e+09,1,None,0.411981,4.423,24,I-24 E,...,121-04193,I-24|I-40,36.151710,-86.75243,36.148640,-86.746120,S,2866076,4326,"LINESTRING (1746358.147 662754.687, 1746358.87..."
26087,9265526,396104093,1.524566e+09,1.524380e+09,1,None,0.457612,5.257,40,I-40 W,...,121-04198,I-40,36.142510,-86.68065,36.143690,-86.688720,W,902228,4326,"LINESTRING (1767527.881 659254.808, 1767411.49..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90681,12945655,1524632690,1.524315e+09,NaN,1,None,0.542552,4.000,65,I-65 S,...,121N04233,I-65,36.228420,-86.77278,36.220890,-86.774700,S,1692224,4326,"LINESTRING (1740581.424 690743.021, 1740339.96..."
90683,12945670,1524632715,1.524610e+09,1.524633e+09,1,None,0.123543,5.093,24,I-24 E,...,"121-04231,121N04231",I-24|I-65,36.208820,-86.77621,36.207040,-86.776420,S,2866076,4326,"LINESTRING (1739508.556 683601.132, 1739490.85..."
90738,12946004,1524633281,NaN,NaN,1,None,0.566494,1.000,24,I-24 E,...,121P10735,I-24,36.131060,-86.72704,36.122980,-86.727730,S,6504651,4326,"LINESTRING (1753803.105 655203.146, 1753789.68..."
90850,12947013,1524635232,NaN,NaN,1,None,0.355533,1.476,,86B,...,121P10823,86B,36.191850,-86.78020,36.188260,-86.776120,S,6549630,4326,"LINESTRING (1738189.735 677443.912, 1738358.21..."


In [11]:
Davidson_incident = gpd.sjoin(incidents_project, DAVIDSON, how='inner', op='intersects')
Davidson_incident

/home/baiting/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,confidence,country,geo,magvar,nThumbsUp,Reliability,reportRating,roadType,street,subtype,...,PrimaryTmc,RoadList,StartLat,StartLong,EndLat,EndLong,Bearing,XDGroup,ShapeSRID,space
8751,5,US,POINT(-86.80638 36.170188),79,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,...,"121+04186,121P04185",I-40,36.166930,-86.816540,36.171460,-86.802770,E,1685035,4326,"LINESTRING (1727480.908 668447.641, 1727633.76..."
2102,0,US,POINT(-86.807995 36.169948),79,0.0,6.0,2,3.0,I-40 E,ACCIDENT_MINOR,...,"121+04186,121P04185",I-40,36.166930,-86.816540,36.171460,-86.802770,E,1685035,4326,"LINESTRING (1727480.908 668447.641, 1727633.76..."
9054,2,US,POINT(-86.813918 36.168637),57,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,...,"121+04186,121P04185",I-40,36.166930,-86.816540,36.171460,-86.802770,E,1685035,4326,"LINESTRING (1727480.908 668447.641, 1727633.76..."
3768,0,US,POINT(-86.815475 36.167764),48,0.0,5.0,1,3.0,I-40 E,ACCIDENT_MINOR,...,"121+04186,121P04185",I-40,36.166930,-86.816540,36.171460,-86.802770,E,1685035,4326,"LINESTRING (1727480.908 668447.641, 1727633.76..."
5935,0,US,POINT(-86.815426 36.1678),48,0.0,5.0,1,3.0,I-40 E,ACCIDENT_MAJOR,...,"121+04186,121P04185",I-40,36.166930,-86.816540,36.171460,-86.802770,E,1685035,4326,"LINESTRING (1727480.908 668447.641, 1727633.76..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49770,0,US,POINT(-86.989866 36.077849),277,NaN,5.0,2,3.0,I-40 E,,...,121+04178,I-40,36.077692,-86.996682,36.077439,-86.986797,E,1685035,4326,"LINESTRING (1673975.548 636471.734, 1674530.29..."
28081,0,US,POINT(-86.992596 36.077881),266,NaN,6.0,5,3.0,I-40 E,,...,121+04178,I-40,36.077692,-86.996682,36.077439,-86.986797,E,1685035,4326,"LINESTRING (1673975.548 636471.734, 1674530.29..."
39987,5,US,POINT(-86.986925 36.077466),107,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,...,121+04178,I-40,36.077692,-86.996682,36.077439,-86.986797,E,1685035,4326,"LINESTRING (1673975.548 636471.734, 1674530.29..."
13391,5,US,POINT(-86.990634 36.077887),92,0.0,10.0,2,3.0,I-40 E,,...,121+04178,I-40,36.077692,-86.996682,36.077439,-86.986797,E,1685035,4326,"LINESTRING (1673975.548 636471.734, 1674530.29..."


In [6]:
#Find the distance between the road segment and the location of accident
acc_point = gpd.GeoSeries.from_wkt(Davidson_incident.geo, crs='EPSG:4326')
Davidson_incident['distance'] = acc_point.to_crs(tnESPG).distance(Davidson_incident.space, align=False)

In [7]:
#Find the segment which is the closest to the accident location
segments = {}
for accident_id, group in Davidson_incident.groupby(by='uuid'):
    segments[accident_id] = str(group[group['distance'] == group['distance'].min()]['XDSegID'].values[0])
Davidson_incident[incidents_project.columns]

,confidence,country,geo,magvar,nThumbsUp,Reliability,reportRating,roadType,street,subtype,ts,type,uuid,ts_utc,dayofweek,original_city,datetime,geometry,geo_buffer
8751,5,US,POINT(-86.80638 36.170188),79,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,2019-04-22 12:06:58 UTC,ACCIDENT,cec392ff-8cfc-3bd0-a287-b08e627462c3,2019-04-22 12:06:58,Mon,"Nashville, TN",2019-04-22 12:06:58 UTC,POINT (1730487.559 669610.502),"POLYGON ((1730507.559 669610.502, 1730507.463 ..."
2102,0,US,POINT(-86.807995 36.169948),79,0.0,6.0,2,3.0,I-40 E,ACCIDENT_MINOR,2019-04-22 12:29:08 UTC,ACCIDENT,9c5a0fe3-e537-34e1-8b9a-e773773b47dc,2019-04-22 12:29:08,Mon,"Nashville, TN",2019-04-22 12:29:08 UTC,POINT (1730010.163 669527.068),"POLYGON ((1730030.163 669527.068, 1730030.067 ..."
9054,2,US,POINT(-86.813918 36.168637),57,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,2019-04-23 13:52:20 UTC,ACCIDENT,2364be5b-78ef-3c91-a771-fba5bf123de4,2019-04-23 13:52:20,Tue,"Nashville, TN",2019-04-23 13:52:20 UTC,POINT (1728257.986 669064.320),"POLYGON ((1728277.986 669064.320, 1728277.890 ..."
3768,0,US,POINT(-86.815475 36.167764),48,0.0,5.0,1,3.0,I-40 E,ACCIDENT_MINOR,2019-04-24 19:23:16 UTC,ACCIDENT,434ea311-4156-3df3-8137-61c157505055,2019-04-24 19:23:16,Wed,"Nashville, TN",2019-04-24 19:23:16 UTC,POINT (1727795.773 668750.353),"POLYGON ((1727815.773 668750.353, 1727815.677 ..."
5935,0,US,POINT(-86.815426 36.1678),48,0.0,5.0,1,3.0,I-40 E,ACCIDENT_MAJOR,2019-04-24 19:28:39 UTC,ACCIDENT,d9154eb5-d70b-34ae-980f-dc3f3bc585f7,2019-04-24 19:28:39,Wed,"Nashville, TN",2019-04-24 19:28:39 UTC,POINT (1727810.346 668763.338),"POLYGON ((1727830.346 668763.338, 1727830.249 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49770,0,US,POINT(-86.989866 36.077849),277,NaN,5.0,2,3.0,I-40 E,,2021-10-29 18:25:26 UTC,ACCIDENT,046a9b3f-6e8b-4cc6-826e-1793ca679a9b,2021-10-29 18:25:26,Fri,"Nashville, TN",2021-10-29 18:25:26 UTC,POINT (1675991.228 636494.923),"POLYGON ((1676011.228 636494.923, 1676011.131 ..."
28081,0,US,POINT(-86.992596 36.077881),266,NaN,6.0,5,3.0,I-40 E,,2021-11-18 12:02:02 UTC,ACCIDENT,0b1d7d9f-2d55-4185-9e7c-92bc6d1f7680,2021-11-18 12:02:02,Thu,"Nashville, TN",2021-11-18 12:02:02 UTC,POINT (1675184.649 636514.742),"POLYGON ((1675204.649 636514.742, 1675204.553 ..."
39987,5,US,POINT(-86.986925 36.077466),107,0.0,10.0,2,3.0,I-40 E,ACCIDENT_MINOR,2021-12-19 00:59:34 UTC,ACCIDENT,764990e1-9fea-433c-b1fc-f55746fb746e,2021-12-19 00:59:34,Sun,"Nashville, TN",2021-12-19 00:59:34 UTC,POINT (1676858.868 636346.730),"POLYGON ((1676878.868 636346.730, 1676878.771 ..."
13391,5,US,POINT(-86.990634 36.077887),92,0.0,10.0,2,3.0,I-40 E,,2021-12-23 01:55:14 UTC,ACCIDENT,4abfbeeb-ab60-4e23-b427-579d390b17e0,2021-12-23 01:55:14,Thu,"Nashville, TN",2021-12-23 01:55:14 UTC,POINT (1675764.429 636511.051),"POLYGON ((1675784.429 636511.051, 1675784.332 ..."


In [8]:
#Locate the roadsegment with accident and make timestamp aligned with traffic dataset
Davidson_incident_final = Davidson_incident[incidents_project.columns].copy()
Davidson_incident_final.drop_duplicates(inplace=True)
Davidson_incident_final.set_index('uuid', inplace=True)
for uuid, row in Davidson_incident_final.iterrows():
    Davidson_incident_final.at[uuid, 'AccXDSegID'] = int(segments[uuid])
Davidson_incident_final.reset_index(inplace=True)
for i, row in Davidson_incident_final.iterrows():
    datetime = list(row['datetime'][:-4])
    datetime[-1] = '0'
    datetime[-2] = '0'
    if int(datetime[-4]) >= 0 and int(datetime[-4]) < 5:
        datetime[-4] = '0'
    else:
        datetime[-4] = '5'
    datetime = ''.join(datetime)
    Davidson_incident_final.loc[i, 'dt'] = pd.to_datetime(datetime)
Davidson_incident_final['ts'] = Davidson_incident_final[['dt']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)
Davidson_incident_final = Davidson_incident_final.sort_values('ts')
Davidson_incident_final.to_csv('accidents_nashville.csv', index=False)
Davidson_incident_final


,uuid,confidence,country,geo,magvar,nThumbsUp,Reliability,reportRating,roadType,street,...,ts,type,ts_utc,dayofweek,original_city,datetime,geometry,geo_buffer,AccXDSegID,dt
0,cec392ff-8cfc-3bd0-a287-b08e627462c3,5,US,POINT(-86.80638 36.170188),79,0.0,10.0,2,3.0,I-40 E,...,1555934700,ACCIDENT,2019-04-22 12:06:58,Mon,"Nashville, TN",2019-04-22 12:06:58 UTC,POINT (1730487.559 669610.502),"POLYGON ((1730507.559 669610.502, 1730507.463 ...",1.524358e+09,2019-04-22 12:05:00
123,e2923910-7a53-3b62-893e-6f0cc55d05f5,0,US,POINT(-86.776231 36.208708),186,0.0,6.0,1,3.0,I-65 S,...,1555934700,ACCIDENT,2019-04-22 12:09:15,Mon,"Nashville, TN",2019-04-22 12:09:15 UTC,POINT (1739497.416 683560.843),"POLYGON ((1739517.416 683560.843, 1739517.320 ...",1.524633e+09,2019-04-22 12:05:00
122,a7062075-2b55-30d5-ba11-82eb9f2f1103,3,US,POINT(-86.776219 36.208805),186,0.0,10.0,1,3.0,I-65 S,...,1555934700,ACCIDENT,2019-04-22 12:08:55,Mon,"Nashville, TN",2019-04-22 12:08:55 UTC,POINT (1739501.236 683596.126),"POLYGON ((1739521.236 683596.126, 1739521.140 ...",1.524633e+09,2019-04-22 12:05:00
493,aa4b1b43-3b3a-3040-8c34-a055a6ebb37c,0,US,POINT(-86.856853 36.321678),107,0.0,6.0,5,3.0,I-24 E,...,1555935300,ACCIDENT,2019-04-22 12:18:58,Mon,"Nashville, TN",2019-04-22 12:18:58 UTC,POINT (1716073.069 724883.954),"POLYGON ((1716093.069 724883.954, 1716092.973 ...",1.524309e+09,2019-04-22 12:15:00
1,9c5a0fe3-e537-34e1-8b9a-e773773b47dc,0,US,POINT(-86.807995 36.169948),79,0.0,6.0,2,3.0,I-40 E,...,1555935900,ACCIDENT,2019-04-22 12:29:08,Mon,"Nashville, TN",2019-04-22 12:29:08 UTC,POINT (1730010.163 669527.068),"POLYGON ((1730030.163 669527.068, 1730030.067 ...",1.524358e+09,2019-04-22 12:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9268,ab9e9892-5f71-406a-a79e-bddd18649768,5,US,POINT(-86.761827 36.163006),139,NaN,10.0,3,3.0,I-24 E,...,1646602200,ACCIDENT,2022-03-06 21:32:43,Sun,"Nashville, TN",2022-03-06 21:32:43 UTC,POINT (1743617.271 666890.720),"POLYGON ((1743637.271 666890.720, 1743637.174 ...",1.524390e+09,2022-03-06 21:30:00
3290,e0e0d3bf-b186-4f66-9b0a-f81938e935cd,5,US,POINT(-86.752368 36.151668),296,0.0,10.0,2,3.0,I-40 E,...,1646602200,ACCIDENT,2022-03-06 21:32:58,Sun,"Nashville, TN",2022-03-06 21:32:58 UTC,POINT (1746377.676 662741.818),"POLYGON ((1746397.676 662741.818, 1746397.579 ...",3.960700e+08,2022-03-06 21:30:00
7211,69d52783-d8c9-4760-b10b-d358c8282fcb,5,US,POINT(-86.76519 36.166961),153,0.0,10.0,5,3.0,I-24 E,...,1646602500,ACCIDENT,2022-03-06 21:38:47,Sun,"Nashville, TN",2022-03-06 21:38:47 UTC,POINT (1742635.828 668338.180),"POLYGON ((1742655.828 668338.180, 1742655.732 ...",1.524494e+09,2022-03-06 21:35:00
11874,422d703a-69a5-4ea8-bf3a-1cc8d1e337d4,2,US,POINT(-86.752502 36.1519),296,NaN,9.0,3,3.0,I-40 W,...,1646602800,ACCIDENT,2022-03-06 21:41:24,Sun,"Nashville, TN",2022-03-06 21:41:24 UTC,POINT (1746338.765 662826.575),"POLYGON ((1746358.765 662826.575, 1746358.669 ...",1.524305e+09,2022-03-06 21:40:00


In [9]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col,when,regexp_replace
import glob
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '64G'), ('spark.driver.memory', '64G'),  ("spark.driver.maxResultSize", '64G')]).setMaster("local[200]")
sc = pyspark.SparkContext(conf=conf)
sparkSession = (SparkSession.builder
  .appName("data_generation")
  .getOrCreate())

filename = './accident_data/test.csv'
files = glob.glob('./traffic/county=Davidson/year=2022/month=1/*.parquet')
agg_df = sparkSession.sparkContext.emptyRDD()
for f in files:
    traffic_pq = sparkSession.read.parquet(f)
    traffic_pq.createOrReplaceTempView("traffic_pq")
traffic = traffic_pq
agg_df_sorted = traffic.sort(traffic.measurement_tstamp.asc())

traffic_df = agg_df_sorted.withColumn('tstamp', unix_timestamp('measurement_tstamp'))

acc = sparkSession.read.option("header", True).csv('accidents_nashville.csv').select('ts','AccXDSegID','type')

22/05/02 11:41:06 WARN Utils: Your hostname, baiting resolves to a loopback address: 127.0.1.1; using 10.66.64.130 instead (on interface wlp62s0)
22/05/02 11:41:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 11:41:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/02 11:41:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/02 11:41:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
cond = [acc.ts == traffic_df.tstamp,
        traffic_df.xd_id == acc.AccXDSegID]

joined_inner = traffic_df.join(acc, cond)

joined_inner = joined_inner.na.fill('0.0', subset=["type"])
accident_final = joined_inner.withColumn('type',
                            when(col('type') == 'ACCIDENT', regexp_replace(joined_inner.type, 'ACCIDENT', '1.0'))
                            .otherwise(joined_inner.type) \
                            )

accident_final.toPandas().to_csv(filename)


In [ ]:
cond = [acc.ts == traffic_df.tstamp,
        traffic_df.xd_id == acc.AccXDSegID]

joined_left = traffic_df.join(acc, cond, 'left')

joined_left = joined_left.na.fill('0.0', subset=["type"])
full_data = joined_left.withColumn('type',
                            when(col('type') == 'ACCIDENT', regexp_replace(joined_left.type, 'ACCIDENT', '1.0'))
                            .otherwise(joined_left.type) \
                            )

full_data.limit(22000).toPandas().to_csv(filename)